# Cancri (scatterplot)

In [2]:
import pandas as pd
import numpy as np
import en_core_web_sm
nlp = en_core_web_sm.load()
%matplotlib inline
import scattertext as st
import time
from scipy.stats import rankdata, hmean, norm
from pprint import pprint
from IPython.display import IFrame
from IPython.core.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer

<br><br><br>En la siguiente celda abrimos el dataset maestro y hacemos merge con el dataset de bandas y géneros.<br>
Un paso intermedio consiste en pasar a mayúsculas los nombres de todas las bandas. <br>
El nuevo dataframe es llamado "new_df" y contiene 1 columna adicional: el género de la banda <br><br><br>

In [5]:
df_en = pd.read_pickle("./archivos_modelos/df_darklyrics_en.pkl")
print (df_en.shape)

(176858, 6)


In [6]:
df_en

,band_name,album_name,album_year,song_title,lyric,lang
0,CHRONIX REVOLT,The Impending Fall Of The System,2014,1. The Impending Fall Of The System,New times Disguise falling Of planners behind ...,en
2,DEFEATED SANITY,Chapters Of Repugnance,2010,2. Consumed By Repugnance,Dismal life of torment Ripping at my body Horr...,en
3,DEFEATED SANITY,Chapters Of Repugnance,2010,3. Carnal Deliverance,In bitterness and repugnance I've passed the p...,en
4,DEFEATED SANITY,Chapters Of Repugnance,2010,4. Salacious Affinity,Beloved children missing Swallowed in obscurit...,en
5,DEFEATED SANITY,Chapters Of Repugnance,2010,5. Engulfed In Excruciation,Destitute and unwanted Departing from asylum L...,en
6,DEFEATED SANITY,Chapters Of Repugnance,2010,6. Coerced Into Idolatry,Urethra corded up (While) wine is infused in l...,en
7,DEFEATED SANITY,Chapters Of Repugnance,2010,7. Blissfully Exsanguinated,Delightfully dripping from above as I wallow i...,en
8,DEFEATED SANITY,Chapters Of Repugnance,2010,8. Calculated Barbarity,Human creativity Spawning amorality Altruism a...,en
9,DEFEATED SANITY,Chapters Of Repugnance,2010,9. Lurid Assimilation,Moments of unspeakable suffering affecting my ...,en
10,DEFEATED SANITY,Psalms Of The Moribund,2007,1. Stoned Then Defiled,Victim Beaten Torn And Raped. Unwanted Interco...,en


In [38]:
df_bands = pd.read_csv("./archivos_modelos/band_name_genre.csv") 

df_bands['band_name'] = df_bands['band_name'].str.upper()
new_df = pd.merge(df_bands,df_en)
new_df.to_pickle("./archivos_modelos/df_en_bygenre.pkl")
print (new_df.head(3))

         genre      band_name  album_name album_year  \
0  Black Metal  ...AND OCEANS  A.M.G.O.D.       2001   
1  Black Metal  ...AND OCEANS  A.M.G.O.D.       2001   
2  Black Metal  ...AND OCEANS  A.M.G.O.D.       2001   

                 song_title  \
0   1. Intelligence Is Sexy   
1  2. White Synthetic Noise   
2     3. Tears Have No Name   

                                               lyric lang  
0  Electro eroticism Intelligence is sexy Electro...   en  
1  Inhale/exhale Inhale the exhaled Inhale/exhale...   en  
2  Under the dark sky we stand Under heavy rain w...   en  


In [35]:
print (df_en.shape)
print (new_df.head(2))
print ('\n', new_df.shape)

(27255, 10)
         genre      band_name  album_name album_year  \
0  Black Metal  ...AND OCEANS  A.M.G.O.D.       2001   
1  Black Metal  ...AND OCEANS  A.M.G.O.D.       2001   

                 song_title  \
0   1. Intelligence Is Sexy   
1  2. White Synthetic Noise   

                                               lyric lang  lyric_len  
0  Electro eroticism Intelligence is sexy Electro...   en        531  
1  Inhale/exhale Inhale the exhaled Inhale/exhale...   en        497  

 (34553, 8)


<br><br><br>
En la siguiente celda creamos una columna adicional que se llama "len" que mide el largo de cada "lyric".<br>
Posteriormente se ordenan por genre, band_name & album_name (ascending).<br><br><br>

In [4]:
new_df["lyric"]= new_df["lyric"].astype(str)
new_df["lyric_len"]= new_df["lyric"].str.len()
print (new_df.sort_values(['genre', 'band_name','album_name'], ascending=[True, True, True]))

             genre      band_name                               album_name  \
15     Black Metal  ...AND OCEANS  ...And Oceans Vs. Bloodthorn WAR Vol. I   
0      Black Metal  ...AND OCEANS                               A.M.G.O.D.   
1      Black Metal  ...AND OCEANS                               A.M.G.O.D.   
2      Black Metal  ...AND OCEANS                               A.M.G.O.D.   
3      Black Metal  ...AND OCEANS                               A.M.G.O.D.   
4      Black Metal  ...AND OCEANS                               A.M.G.O.D.   
5      Black Metal  ...AND OCEANS                               A.M.G.O.D.   
6      Black Metal  ...AND OCEANS                               A.M.G.O.D.   
16     Black Metal  ...AND OCEANS                                   Cypher   
17     Black Metal  ...AND OCEANS                                   Cypher   
18     Black Metal  ...AND OCEANS                                   Cypher   
19     Black Metal  ...AND OCEANS                               

In [5]:
print (new_df['genre'].value_counts())

Power Metal    13636
Black Metal    13619
Heavy Metal     5185
Glam Metal      2113
Name: genre, dtype: int64


<br><br><br>Power Metal    13636 <br>
Black Metal    13619<br>
Heavy Metal     5185<br>
Glam Metal      2113<br><br><br>

In [9]:
power = new_df.loc[new_df['genre'] == 'Power Metal']
black = new_df.loc[new_df['genre'] == 'Black Metal']
heavy = new_df.loc[new_df['genre'] == 'Heavy Metal']
glam = new_df.loc[new_df['genre'] == 'Glam Metal']

power_black = pd.concat([power, black])

print (power_black.shape)
print (power_black.head(2))

(27255, 8)
          genre          band_name           album_name album_year  \
79  Power Metal  3 INCHES OF BLOOD  Here Waits Thy Doom       2009   
80  Power Metal  3 INCHES OF BLOOD  Here Waits Thy Doom       2009   

                    song_title  \
79  1. Battles And Brotherhood   
80             2. Rock In Hell   

                                                lyric lang  lyric_len  
79  With battleaxes drawn we race across the sky H...   en       1296  
80  Across the seas they come Great bard trek from...   en        940  


In [12]:
power_black.groupby('genre').apply(lambda x: x.lyric.apply(lambda x: len(x.split())).sum())
power_black['parsed'] = power_black.lyric.apply(nlp)
power_black.to_pickle("./archivos_modelos/df_en_power_black_tknz.pkl") #guardar el dataset con la columna extra de tokens 

In [13]:
power_black = pd.read_pickle("./archivos_modelos/df_en_power_black_tknz.pkl")
print (power_black)

             genre          band_name                    album_name  \
79     Power Metal  3 INCHES OF BLOOD           Here Waits Thy Doom   
80     Power Metal  3 INCHES OF BLOOD           Here Waits Thy Doom   
81     Power Metal  3 INCHES OF BLOOD           Here Waits Thy Doom   
82     Power Metal  3 INCHES OF BLOOD           Here Waits Thy Doom   
83     Power Metal  3 INCHES OF BLOOD           Here Waits Thy Doom   
84     Power Metal  3 INCHES OF BLOOD           Here Waits Thy Doom   
85     Power Metal  3 INCHES OF BLOOD           Here Waits Thy Doom   
86     Power Metal  3 INCHES OF BLOOD           Here Waits Thy Doom   
87     Power Metal  3 INCHES OF BLOOD           Here Waits Thy Doom   
88     Power Metal  3 INCHES OF BLOOD           Here Waits Thy Doom   
89     Power Metal  3 INCHES OF BLOOD  Battlecry Under A Winter Sun   
90     Power Metal  3 INCHES OF BLOOD  Battlecry Under A Winter Sun   
91     Power Metal  3 INCHES OF BLOOD  Battlecry Under A Winter Sun   
92    

In [14]:
corpus = st.CorpusFromParsedDocuments(power_black, category_col='genre', parsed_col='parsed').build()

In [15]:
term_freq_df = corpus.get_term_freq_df()
#print (term_freq_df)
term_freq_df['black_precision'] = term_freq_df['Black Metal freq'] * 1./(term_freq_df['Black Metal freq'] + term_freq_df['Power Metal freq'])
#print (term_freq_df['black_precision'])
term_freq_df['black_freq_pct'] = term_freq_df['Black Metal freq'] * 1./term_freq_df['Black Metal freq'].sum()
#print (term_freq_df['black_freq_pct'])
term_freq_df['black_hmean'] = term_freq_df.apply(lambda x: (hmean([x['black_precision'], x['black_freq_pct']])
                                                                   if x['black_precision'] > 0 and x['black_freq_pct'] > 0 
                                                                   else 0), axis=1)
#print ('\n\n\n\n\n\n\n\nnext\n\n\n\n\n\n\n\n')
term_freq_df.sort_values(by='black_hmean', ascending=False).iloc[:10]



,Power Metal freq,Black Metal freq,black_precision,black_freq_pct,black_hmean
term,,,,,
the,168904,153807,0.476609,0.036496,0.067800
of,59551,76084,0.560947,0.018053,0.034981
and,54957,46703,0.459404,0.011082,0.021642
i,79076,45851,0.367022,0.010880,0.021133
to,65272,45011,0.408141,0.010680,0.020816
in,44708,38541,0.462961,0.009145,0.017936
a,46548,32902,0.414122,0.007807,0.015325
you,63571,28407,0.308846,0.006740,0.013193
my,37216,26499,0.415899,0.006288,0.012388


In [18]:
def normcdf(x):
    return norm.cdf(x, x.mean(), x.std())
term_freq_df['black_precision_normcdf'] = normcdf(term_freq_df['black_precision'])
term_freq_df['black_freq_pct_normcdf'] = normcdf(term_freq_df['black_freq_pct'])
term_freq_df['black_scaled_f_score'] = hmean([term_freq_df['black_precision_normcdf'], term_freq_df['black_freq_pct_normcdf']])
term_freq_df.sort_values(by='black_scaled_f_score', ascending=False).iloc[:100]

,Power Metal freq,Black Metal freq,black_precision,black_freq_pct,black_hmean,black_precision_normcdf,black_freq_pct_normcdf,black_scaled_f_score
term,,,,,,,,
ov,0,507,1.000000,0.000120,0.000241,0.816920,0.989426,0.894935
of satan,22,538,0.960714,0.000128,0.000255,0.793103,0.992807,0.881789
_,0,351,1.000000,0.000083,0.000167,0.816920,0.944070,0.875905
og,4,361,0.989041,0.000086,0.000171,0.810454,0.949054,0.874295
satan,255,2245,0.898000,0.000533,0.001065,0.751501,1.000000,0.858122
som,20,348,0.945652,0.000083,0.000165,0.783505,0.942503,0.855681
blasphemy,43,418,0.906725,0.000099,0.000198,0.757543,0.971097,0.851129
der,6,278,0.978873,0.000066,0.000132,0.804331,0.895280,0.847373
nocturnal,28,345,0.924933,0.000082,0.000164,0.769892,0.940900,0.846849


In [20]:
term_freq_df['black_corner_score'] = corpus.get_corner_scores('Black Metal')
term_freq_df.sort_values(by='black_corner_score', ascending=False).iloc[:100]

,Power Metal freq,Black Metal freq,black_precision,black_freq_pct,black_hmean,black_precision_normcdf,black_freq_pct_normcdf,black_scaled_f_score,black_corner_score
term,,,,,,,,,
ov,0,507,1.0,0.000120,0.000241,0.81692,0.989426,0.894935,0.910338
_,0,351,1.0,0.000083,0.000167,0.81692,0.944070,0.875905,0.910337
satanas,0,198,1.0,0.000047,0.000094,0.81692,0.812790,0.814849,0.910336
|,0,173,1.0,0.000041,0.000082,0.81692,0.780408,0.798246,0.910335
_ _,0,161,1.0,0.000038,0.000076,0.81692,0.763787,0.789461,0.910334
for satan,0,101,1.0,0.000024,0.000048,0.81692,0.671253,0.736957,0.910328
abigail,0,93,1.0,0.000022,0.000044,0.81692,0.657881,0.728825,0.910326
skal,0,91,1.0,0.000022,0.000043,0.81692,0.654506,0.726749,0.910325
jeg,0,86,1.0,0.000020,0.000041,0.81692,0.646015,0.721484,0.910324


In [23]:
term_freq_df = corpus.get_term_freq_df()
term_freq_df['Power Metal Score'] = corpus.get_scaled_f_scores('Power Metal')
term_freq_df['Black Metal Score'] = corpus.get_scaled_f_scores('Black Metal')
print("Top 10 Black Metal terms")
pprint(list(term_freq_df.sort_values(by='Black Metal Score', ascending=False).index[:100]))
print("Top 10 Power Metal terms")
pprint(list(term_freq_df.sort_values(by='Power Metal Score', ascending=False).index[:100]))

Top 10 Black Metal terms
['ov',
 'of satan',
 'satan',
 'infernal',
 'thou',
 'forth',
 'christ',
 'worship',
 'blasphemy',
 'fucking',
 'og',
 '_',
 '/',
 'funeral',
 'horns',
 'thee',
 'serpent',
 'christian',
 'abyss',
 'existence',
 'flesh',
 'lucifer',
 'som',
 'thy',
 'art',
 'o',
 'en',
 "satan 's",
 'which',
 'nocturnal',
 'wolves',
 'upon the',
 'torment',
 'of god',
 'soil',
 'storms',
 'behold',
 'whore',
 'corpse',
 'total',
 'wrath',
 'the flesh',
 'forest',
 'goat',
 'the abyss',
 'jesus',
 'unto',
 'of blood',
 'unholy',
 'fog',
 'den',
 'of death',
 'chaos',
 'tongue',
 'north',
 'satanic',
 'realm',
 'rape',
 'och',
 'shall be',
 'shall',
 'lust',
 'upon',
 'i shall',
 'form',
 'death',
 'ancient',
 'throne',
 'pale',
 'the ancient',
 'mist',
 '&',
 'towards',
 'blood of',
 'hatred',
 '& amp',
 'amp',
 'der',
 'plague',
 'trees',
 'of christ',
 'depths',
 'the blood',
 'woods',
 'summon',
 'damnation',
 'frost',
 'lips',
 '-',
 'cursed',
 'of darkness',
 'fuck',
 'rite

# Visualización

<br><br><br>
- Una palabra usada 10 veces en el Power Metal, estará en la posición 10 en el eje x<br>
- Esto no es muy útil. Todo, excepto los términos más frecuentes, quedan apiñados en la esquina inferior-izquierda.<br>
- Los puntajes de corner-distance son mayormente stopwords<br>
- Por defecto, el color de las palabras representa el puntaje F escalado (Scaled F-Score)<br><br><br>

In [24]:
html = produce_scattertext_explorer(corpus,
                                    category='Black Metal',
                                    category_name='Black Metal',
                                    not_category_name='Power Metal',
                                    width_in_pixels=1000,
                                    minimum_term_frequency=5,
                                    transform=st.Scalers.scale,
                                    metadata=power_black['band_name'])
file_name = './power_black_v1_ugly.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1152, height=864)

<br><br><br>
Rankear términos por sus percentiles de frecuencia en vez de la frecuencia bruta. <br>
Un término en la mitad del eje x será mencionado en el Power Metal con una frecuencia media. <br>
Esto distribuye los términos de manera más elegante en el espacio. <br>
Haciendo solamente eso, los términos con las mismas frecuencias en las 2 clases, quedarían solapados entre sí. <br>
El punto que se vería sería el primero y los demás quedarían "detrás" de éste impidiendo hacer mouseover.<br>
La solución preferida es llevarlos a orden alfabético entre términos de igual frecuencia<br><br><br>

In [25]:
html = produce_scattertext_explorer(corpus,
                                    category='Black Metal',
                                    category_name='Black Metal',
                                    not_category_name='Power Metal',
                                    width_in_pixels=1000,
                                    minimum_term_frequency=5,
                                    metadata=power_black['band_name'],
                                    term_significance = st.LogOddsRatioUninformativeDirichletPrior())
file_name = './power_black_v2_pretty.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1152, height=864)

<br><br><br>Visualización de los coeficientes de regresión L2-penalizados v/s log-frecuencia de términos. <br><br><br>


In [27]:
def scale(ar): 
    return (ar - ar.min()) / (ar.max() - ar.min())

def zero_centered_scale(ar):
    scores = np.zeros(len(ar))
    scores[ar > 0] = scale(ar[ar > 0])
    scores[ar < 0] = -scale(-ar[ar < 0])
    return (scores + 1) / 2.

frequencies_scaled = scale(np.log(term_freq_df.sum(axis=1).values))

In [29]:
from sklearn.linear_model import LogisticRegression
scores = corpus.get_logreg_coefs('Black Metal',
                                 LogisticRegression(penalty='l2', C=10, max_iter=10000, n_jobs=-1))
scores_scaled = zero_centered_scale(scores)

html = produce_scattertext_explorer(corpus,
                                    category='Black Metal',
                                    category_name='Black Metal',
                                    not_category_name='Power Metal',
                                    minimum_term_frequency=5,
                                    width_in_pixels=1000,
                                    x_coords=frequencies_scaled,
                                    y_coords=scores_scaled,
                                    scores=scores,
                                    sort_by_dist=False,
                                    metadata=power_black['band_name'],
                                    x_label='Log frequency',
                                    y_label='L2-Penalized Log Reg Coef')
file_name = './L2vsLog_pretty.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

<br><br><br>Comparación del método anterior con el método de F-escalado.<br><br><br>

In [30]:
html = produce_scattertext_explorer(corpus,
                                    category='Black Metal',
                                    category_name='Black Metal',
                                    not_category_name='Power Metal',
                                    minimum_term_frequency=5,
                                    width_in_pixels=1000,
                                    x_coords=frequencies_scaled,
                                    y_coords=corpus.get_scaled_f_scores('Black Metal', beta=0.5),
                                    scores=corpus.get_scaled_f_scores('Black Metal', beta=0.5),
                                    sort_by_dist=False,
                                    metadata=power_black['band_name'],
                                    x_label='Log Frequency',
                                    y_label='Scaled F-Score')
file_name = './power_black_SFSvsLog_pretty.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

<br><br><br>
Una forma de visualizar es usando la razón de log-probabilidad penalizada (penalized log-odds-ratio).
<br>
Burt L. Monroe, Michael P. Colaresi, and Kevin M. Quinn. 2008. Fightin’ words: Lexical feature selection and evaluation for identifying the content of political conflict. Political Analysis.<br><br><br>

In [32]:
freq_df = corpus.get_term_freq_df().rename(columns={'Black Metal freq': 'y_bl', 'Power Metal freq': 'y_pw'})
a_w = 0.01
y_i, y_j = freq_df['y_bl'].values, freq_df['y_pw'].values

In [33]:
n_i, n_j = y_i.sum(), y_j.sum()
a_0 = len(freq_df) * a_w
delta_i_j = (  np.log((y_i + a_w) / (n_i + a_0 - y_i - a_w))
                 - np.log((y_j + a_w) / (n_j + a_0 - y_j - a_w)))
var_delta_i_j = ( 1./(y_i + a_w) + 1./(y_i + a_0 - y_i - a_w)
                    + 1./(y_j + a_w) + 1./(n_j + a_0 - n_j - a_w))
zeta_i_j = delta_i_j/np.sqrt(var_delta_i_j)
max_abs_zeta = max(zeta_i_j.max(), -zeta_i_j.min())
zeta_scaled_for_charting = ((((zeta_i_j > 0).astype(float) * (zeta_i_j/max_abs_zeta))*0.5 + 0.5)
                            + ((zeta_i_j < 0).astype(float) * (zeta_i_j/max_abs_zeta) * 0.5))

In [34]:
html = produce_scattertext_explorer(corpus,
                                    category='Black Metal',
                                    category_name='Black Metal',
                                    not_category_name='Power Metal',
                                    minimum_term_frequency=5,
                                    width_in_pixels=1000,
                                    x_coords=frequencies_scaled,
                                    y_coords=zeta_scaled_for_charting,
                                    scores=zeta_i_j,
                                    sort_by_dist=False,
                                    metadata=power_black['band_name'],
                                    x_label='Log Frequency',
                                    y_label='Log Odds Ratio w/ Uninformative Prior (alpha_w=0.01)')
file_name = './power_black_LOPriorvsLog_pretty.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

<br><br><br>El último de la ronda es con los puntajes "corner scores"<br><br><br>
_todavía no lo ejecuto!!!!_

In [ ]:
corner_scores = corpus.get_corner_scores('Black Metal')
html = produce_scattertext_explorer(corpus,
                                    category='Black Metal',
                                    category_name='Black Metal',
                                    not_category_name='Power Metal',
                                    minimum_term_frequency=5,
                                    width_in_pixels=1000,
                                    x_coords=frequencies_scaled,
                                    y_coords=corner_scores,
                                    scores=corner_scores,
                                    sort_by_dist=False,
                                    metadata=power_black['band_name'],
                                    x_label='Log Frequency',
                                    y_label='Corner Scores')
file_name = './power_black_CornervsLog_pretty.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)